## SVM Model for Kepler Space Telescope Data

The purpose of this project is to reinforce my data science and machine learning knowledge, and to help others in their journeys. I'm not an expert on these topics, and I'm conscious that there are a lot of tools, concepts and techniques that I need to master. That's why I develop this project, to share with the community the tools, concepts and techniques, that I have learned across my data science journey. I'm open to comments, critics and feedback that would help me to develop learn best practices and corrections in case I was wrong

In [51]:
#import warnings
#warnings.simplefilter('ignore')
import pandas as pd
import numpy as np

In [52]:
kepler_df = pd.read_csv('../../data/exoplanet_data.csv')
kepler_df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [53]:
kepler_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6991 entries, 0 to 6990
Data columns (total 41 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   koi_disposition    6991 non-null   object 
 1   koi_fpflag_nt      6991 non-null   int64  
 2   koi_fpflag_ss      6991 non-null   int64  
 3   koi_fpflag_co      6991 non-null   int64  
 4   koi_fpflag_ec      6991 non-null   int64  
 5   koi_period         6991 non-null   float64
 6   koi_period_err1    6991 non-null   float64
 7   koi_period_err2    6991 non-null   float64
 8   koi_time0bk        6991 non-null   float64
 9   koi_time0bk_err1   6991 non-null   float64
 10  koi_time0bk_err2   6991 non-null   float64
 11  koi_impact         6991 non-null   float64
 12  koi_impact_err1    6991 non-null   float64
 13  koi_impact_err2    6991 non-null   float64
 14  koi_duration       6991 non-null   float64
 15  koi_duration_err1  6991 non-null   float64
 16  koi_duration_err2  6991 

In [54]:
kepler_df.columns

Index(['koi_disposition', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag'],
      dtype='object')

In [55]:
print(f"The dataframe has a length of {len(kepler_df.columns)} columns.")
print(f"There are 3 outcomes/predictions/targets: {set(kepler_df['koi_disposition'])}")

The dataframe has a length of 41 columns.
There are 3 outcomes/predictions/targets: {'CONFIRMED', 'CANDIDATE', 'FALSE POSITIVE'}


In [56]:
features = kepler_df.drop('koi_disposition', axis = 1)
target = kepler_df['koi_disposition'].values

print(features.shape)
print(target.shape)

(6991, 40)
(6991,)


In [57]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.3, random_state = 42)

In [58]:
print('Training Features Shape:', X_train.shape)
print('Training Labels Shape:', y_train.shape)
print('Testing Features Shape:', X_test.shape)
print('Testing Labels Shape:', y_test.shape)

Training Features Shape: (4893, 40)
Training Labels Shape: (4893,)
Testing Features Shape: (2098, 40)
Testing Labels Shape: (2098,)


In [59]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
label_encoder.fit(target)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
categories = label_encoder.inverse_transform([0, 1, 2])

In [60]:
from sklearn.preprocessing import MinMaxScaler
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [61]:
from sklearn.svm import SVC 
svm = SVC(kernel='linear')
svm.fit(X_train_scaled, encoded_y_train)

SVC(kernel='linear')

In [62]:
prediction = svm.predict(X_test_scaled)

In [63]:
scaled_df = pd.DataFrame({'Prediction':prediction, 'Actual':encoded_y_test}).reset_index(drop=True)
scaled_df.head()

,Prediction,Actual
0,2,2
1,0,0
2,2,2
3,2,2
4,2,2


In [64]:
print(f"SVM Scaled Score: {round(svm.score(X_test_scaled, encoded_y_test)*100,2)}%")

SVM Scaled Score: 84.18%


In [65]:
from sklearn.feature_selection import RFE

important_features = RFE(svm, n_features_to_select=15, step=1)
important_features = important_features.fit(X_train_scaled, encoded_y_train)
new_features = features.loc[:, important_features.support_]

In [66]:
new_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_time0bk_err1,koi_time0bk_err2,koi_impact_err1,koi_duration,koi_duration_err1,koi_duration_err2,koi_steff,koi_steff_err1,koi_steff_err2,koi_slogg,koi_slogg_err2
0,0,0,0,0,0.003520,-0.003520,0.059,4.50700,0.11600,-0.11600,5455,81,-81,4.467,-0.096
1,0,1,0,0,0.000581,-0.000581,5.126,1.78220,0.03410,-0.03410,5853,158,-176,4.544,-0.176
2,0,1,0,0,0.000115,-0.000115,0.115,2.40641,0.00537,-0.00537,5805,157,-174,4.564,-0.168
3,0,0,0,0,0.001130,-0.001130,0.235,1.65450,0.04200,-0.04200,6031,169,-211,4.438,-0.210
4,0,0,0,0,0.001900,-0.001900,0.139,3.14020,0.06730,-0.06730,6046,189,-232,4.486,-0.229


In [67]:
X_train, X_test, y_train, y_test = train_test_split(new_features, target, test_size = 0.3, random_state = 42)

In [68]:
label_encoder = LabelEncoder()
label_encoder.fit(target)
encoded_y_train = label_encoder.transform(y_train)
encoded_y_test = label_encoder.transform(y_test)
categories = label_encoder.inverse_transform([0, 1, 2])

In [69]:
X_scaler = MinMaxScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [70]:
svm.fit(X_train_scaled, encoded_y_train)

SVC(kernel='linear')

In [71]:
if_prediction = svm.predict(X_test_scaled)

In [72]:
filtered_scaled_df = pd.DataFrame({'Prediction':if_prediction, 'Actual':encoded_y_test}).reset_index(drop=True)
filtered_scaled_df.head(10)

,Prediction,Actual
0,2,2
1,0,0
2,2,2
3,2,2
4,2,2
5,1,1
6,1,0
7,0,0
8,0,0
9,2,2


In [73]:
print(f"Filtered and Scaled SVM Model Score: {round(svm.score(X_test_scaled, encoded_y_test)*100,2)}%")

Filtered and Scaled SVM Model Score: 84.18%


In [74]:
parameters = {"C":[1,5,10],
             "gamma": [0.0001, 0.001, 0.01]}

In [75]:
from sklearn.model_selection import GridSearchCV
svc_cv = GridSearchCV(svm, param_grid = parameters, verbose = 3)

In [76]:
svc_cv.fit(X_train_scaled, encoded_y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV 1/5] END .................C=1, gamma=0.0001;, score=0.865 total time=   0.6s
[CV 2/5] END .................C=1, gamma=0.0001;, score=0.830 total time=   0.6s
[CV 3/5] END .................C=1, gamma=0.0001;, score=0.855 total time=   0.5s
[CV 4/5] END .................C=1, gamma=0.0001;, score=0.834 total time=   0.5s
[CV 5/5] END .................C=1, gamma=0.0001;, score=0.836 total time=   0.7s
[CV 1/5] END ..................C=1, gamma=0.001;, score=0.865 total time=   0.5s
[CV 2/5] END ..................C=1, gamma=0.001;, score=0.830 total time=   0.5s
[CV 3/5] END ..................C=1, gamma=0.001;, score=0.855 total time=   0.5s
[CV 4/5] END ..................C=1, gamma=0.001;, score=0.834 total time=   0.6s
[CV 5/5] END ..................C=1, gamma=0.001;, score=0.836 total time=   0.6s
[CV 1/5] END ...................C=1, gamma=0.01;, score=0.865 total time=   0.6s
[CV 2/5] END ...................C=1, gamma=0.01;,

GridSearchCV(estimator=SVC(kernel='linear'),
             param_grid={'C': [1, 5, 10], 'gamma': [0.0001, 0.001, 0.01]},
             verbose=3)

In [77]:
predict = svc_cv.predict(X_test_scaled)
predict

array([2, 0, 2, ..., 1, 1, 0])

In [78]:
pd.DataFrame({'Prediction':predict, 'Actual':encoded_y_test}).reset_index(drop=True).head(10)

,Prediction,Actual
0,2,2
1,0,0
2,2,2
3,2,2
4,2,2
5,1,1
6,1,0
7,0,0
8,0,0
9,2,2


In [79]:
from sklearn.metrics import classification_report, confusion_matrix
print("=======================================")
print("Models Accuracy")
print("=======================================")
print(f"Accuracy: {round(svc_cv.score(X_test_scaled, encoded_y_test)*100,2)}%")
print("---------------------------------------")
print("Classification Report")
print("=======================================")
print(classification_report(encoded_y_test, predict, target_names=categories))
print("---------------------------------------")
print("Best Parameters")
print("=======================================")
print("Tuned Model Parameters: {}".format(svc_cv.best_params_))
print("---------------------------------------")
print("Confusion Matrix")
print("=======================================")
print(confusion_matrix(encoded_y_test, predict))
print("---------------------------------------")

Models Accuracy
Accuracy: 86.51%
---------------------------------------
Classification Report
                precision    recall  f1-score   support

     CANDIDATE       0.80      0.62      0.70       504
     CONFIRMED       0.72      0.85      0.78       574
FALSE POSITIVE       0.98      1.00      0.99      1020

      accuracy                           0.87      2098
     macro avg       0.83      0.82      0.82      2098
  weighted avg       0.87      0.87      0.86      2098

---------------------------------------
Best Parameters
Tuned Model Parameters: {'C': 10, 'gamma': 0.0001}
---------------------------------------
Confusion Matrix
[[ 311  187    6]
 [  76  486   12]
 [   2    0 1018]]
---------------------------------------
